In [1]:
import os
import torch as tc
import numpy as np
from tqdm import tqdm

from star.star import STAR

from torch.autograd import Variable
from pytorch3d.loss import point_mesh_face_distance
from pytorch3d.structures import Meshes, Pointclouds
from curve_utils import CurveUtils
from mesh_manipulation import load_mesh, load_template, save_obj

device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

# functions

In [2]:
def zero_poses(poses):
    poses = poses.clone().detach()
    poses[:,12:17] = 0
    poses[:,54:65] = 0
    poses = Variable(poses, requires_grad=True)
    return poses

# fiting

In [3]:
gender_betas = {}
gender_poses = {}
gender_trans = {}

list_of_loss = {
    "male": {},
    "female": {}
}

for gender in ['female', 'male']:

    # loading bodies
    files_path = f"./data/MOVE4D/{gender}/"
    files = os.listdir(files_path)
    files.sort()

    all_vertices = []
    all_faces = []
    files_iterator = tqdm(files, desc=f"loading {gender} bodies", position=0)
    for file in files_iterator:
        body_vertices = load_mesh(files_path + file, device)
        body_vertices *= 0.001
        body_vertices -= body_vertices.mean(axis=0)
        all_vertices.append(body_vertices)

    body = Pointclouds(points=all_vertices)[0]

    for i in range(10):

        list_of_loss[gender][i] = []

        # training model
        nbetas = 0
        learning_rate = 0.02
        epochs = 5000
        step = 250

        poses = tc.FloatTensor(tc.zeros((1, 72))).to(device)
        poses = Variable(poses, requires_grad=True)
        betas = tc.FloatTensor(tc.zeros((1, nbetas))).to(device)
        betas = Variable(betas, requires_grad=True)
        trans = tc.FloatTensor(tc.zeros((1, 3))).to(device)
        trans = Variable(trans, requires_grad=True)
        star = STAR(gender=gender, num_betas=nbetas)
        star_faces = star.faces[None,...].to(device)

        optimizer = tc.optim.Adam([trans, betas, poses], lr=learning_rate)
        epochs_iterator = tqdm(range(epochs), desc=f"optimizing - loss: inf")
        for it in epochs_iterator:

            if it == 20:
                poses = zero_poses(poses)

            if it%step == 0:
                if nbetas < 300:
                    nbetas += 100
                    new_betas = tc.FloatTensor(np.zeros((1, nbetas))).to(device)
                    new_betas[:,:nbetas-100] = betas
                    betas = Variable(new_betas, requires_grad=True)
                    star = STAR(gender=gender, num_betas=nbetas)
                optimizer = tc.optim.Adam([trans, betas, poses], lr=learning_rate)
                
            d = star(betas=betas, pose=poses, trans=trans)
            optimizer.zero_grad()
            star_meshes = Meshes(verts=d, faces=star_faces)
            loss = point_mesh_face_distance(star_meshes, body)
            loss.backward(retain_graph=True)
            epochs_iterator.set_description(f"optimizing - loss: {loss}")
            optimizer.step()
            elapsed_time = epochs_iterator.format_dict['elapsed']
            if loss < 2e-5:
                break
            if elapsed_time > 600:
                break
            
            list_of_loss[gender][i].append(loss.detach().cpu().numpy())

loading female bodies: 100%|████████████████████| 72/72 [00:08<00:00,  8.71it/s]
optimizing - loss: 0.000781196285970509:   3%| | 127/5000 [00:21<13:30,  6.01it/


KeyboardInterrupt: 

In [ ]:
import pandas as pd
loss_df = pd.DataFrame.from_dict(list_of_loss, orient='index').T

In [ ]:
loss_df.astype(float).plot(logy=True)

In [ ]:
loss_df.to_csv("status_data_3.csv")